In [1]:
# standard dependencies
import numpy as np
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# local package dependencies
sys.path.append(os.path.abspath("../../scripts/"))
sys.path.append(os.path.abspath("../../visualizations/"))

# for making direct query requests to the data warehouse
from oso_db import execute_query

In [3]:
# create a mapping of projects to collections
result = execute_query("""
    SELECT p."slug", c."slug" 
    FROM project p 
    JOIN collection_projects_project cpp ON p."id" = cpp."projectId"
    JOIN collection c ON cpp."collectionId" = c."id"
    WHERE c."typeId" = 1
""", col_names=False)
projects_to_collections = {
    k: (
        sorted([
            v for v, key
            in result
            if key == k
        ]) 
    )
    for k in set(key for _, key in result)
}

In [4]:
# create a list of collections (ecosystems)
collection_slugs = list(projects_to_collections.keys())

In [5]:
# create a list of projects
project_slugs = set([
    p for c,ps 
    in projects_to_collections.items() 
    if c in collection_slugs 
    for p in ps
])
len(project_slugs)

1214

In [38]:
# filter on the project slugs we're interested in
my_slugs = projects_to_collections['octant-02']

In [41]:
# create a string version of the projects list for use in querystrings
slugs_str = "','".join(my_slugs)
slugs_str

"brightid','clrfund','commonsbuild','dao-drops-dorgtech','drips-network','ethereum-cat-herders','ethstaker','gitcoin','givepraise','giveth','glo-foundation','hausdao','hypercerts','kernel-community','l2beat','metagame-metafam','onion-dao','opensource-observer','pairwise-general-magic','protocol-guild','revoke-cash','rotki','shutter-network','tor-project"

In [62]:
# get the names of those projects (up to 24 characters)
result = execute_query(f"""
    SELECT slug, name
    FROM project 
    WHERE slug IN ('{slugs_str}') 
""")
slugs_to_names = {x[0]: x[1][:25] for x in result}

In [44]:
# get data about active developers
query = execute_query(f"""
    SELECT
        p."slug",
        a."name" AS "repo",
        e."fromId",
        et."name",
        e."time",
        e."amount"
    FROM event e             
    JOIN project_artifacts_artifact paa ON e."toId" = paa."artifactId"            
    JOIN artifact a ON paa."artifactId" = a."id"
    JOIN project p ON paa."projectId" = p.id
    JOIN event_type et ON e."typeId" = et."id"
    WHERE
        e."typeId" IN (2,3,4,6,18,14,22)
        AND p.slug IN ('{slugs_str}')
""", col_names=True)

df = pd.DataFrame(query[1:], columns=query[0])
df.head()

,slug,repo,fromId,name,time,amount
0,tor-project,torproject/webwml,33943.0,COMMIT_CODE,2007-12-31 20:38:23+00:00,1.0
1,tor-project,torproject/webwml,150563.0,COMMIT_CODE,2008-01-01 02:35:02+00:00,1.0
2,tor-project,torproject/webwml,150563.0,COMMIT_CODE,2008-01-01 03:09:22+00:00,1.0
3,tor-project,torproject/webwml,150563.0,COMMIT_CODE,2008-01-01 03:22:04+00:00,1.0
4,tor-project,torproject/webwml,150563.0,COMMIT_CODE,2008-01-01 04:17:09+00:00,1.0


In [45]:
df['month'] = df['time'].apply(lambda x: f"{x.year}-{str(x.month).zfill(2)}")
df['day'] = df['time'].apply(lambda x: f"{x.year}-{str(x.month).zfill(2)}-{str(x.day).zfill(2)}")
df['years_ago'] = (df['time'].max() - df['time']).apply(lambda x: x.days / 365.25)

In [46]:
date_thresh = '2023-07-01'

In [63]:
first_pr = (
    df.query("name == 'PULL_REQUEST_CREATED'")
    .groupby('slug')['years_ago']
    .max()
    .apply(lambda x: round(x,1))
    .rename('First PR Created - Years Ago')
)

In [64]:
active_repos = (
    df.query("name != 'FORK_AGGREGATE_STATS' and name != 'STAR_AGGREGATE_STATS'")
    .query("day >= @date_thresh")
    .groupby('slug')['repo']
    .nunique()
    .rename('Active Repos - Last 6 Months')
)

In [65]:
stars = (
    df.query("name == 'STAR_AGGREGATE_STATS'")
    .groupby('slug')['amount']
    .max()
    .rename('Max Stars - Any Repo')
    .astype(int)
)
forks = (
    df.query("name == 'FORK_AGGREGATE_STATS'")
    .groupby('slug')['amount']
    .max()
    .rename('Max Forks - Any Repo')
    .astype(int)
)

In [66]:
repo_activity = (
    df.query("name != 'FORK_AGGREGATE_STATS' and name != 'STAR_AGGREGATE_STATS'")
    .query("day >= @date_thresh")
    .groupby(['slug', 'name'])['amount']
    .sum()
    .sort_values()
    .reset_index()
    .pivot_table(index=['slug'], columns='name', values='amount', fill_value=0)
    .astype(int)
)
repo_activity.columns = [c.replace("_"," ").title() + " - All Repos, Last 6 Months" for c in repo_activity.columns]

In [67]:
contribs = (
    df.query("name == 'COMMIT_CODE' or name == 'PULL_REQUEST_CREATED'")
    .groupby(['slug', 'fromId'])
    .agg({'amount': 'sum', 'time': ['min', 'max']})
)
contribs.columns = ['amount', 'first', 'last']
new_contribs = (
    contribs[(contribs['amount'] > 1) & (contribs['first'] >= date_thresh)]
    .reset_index()
    .groupby('slug')['fromId']
    .nunique()
    .astype(int)
    .rename('New Contributors - Last 6 Months')
)
lifetime_contribs = (
    contribs[contribs['amount'] > 1]
    .reset_index()
    .groupby('slug')['fromId']
    .nunique()
    .astype(int)
    .rename('Total Contributors - All Time')
)

In [68]:
devs = (
    df.query("name == 'COMMIT_CODE' or name == 'PULL_REQUEST_CREATED'")
    .query("day >= @date_thresh")
    .groupby(['slug', 'fromId', 'month'])['day']
    .nunique()
    .rename('activeDays')
    .reset_index()
)
devs['devType'] = devs['activeDays'].apply(lambda x: "Full-time" if x >=10 else "Part-time")
dev_types = (
    (
        devs
        .groupby(['slug', 'devType'])['fromId']
        .count() / 6        
    )
    .reset_index()
    .pivot_table(
        index='slug', 
        columns='devType', 
        values='fromId', 
        fill_value=0
    )
    .rename(columns={
        'Full-time': 'Full-time Developers - Avg Last 6 Months',
        'Part-time': 'Part-time Developers - Avg Last 6 Months'
    })
    .join(
        (
            devs
            .groupby(['slug', 'month'])['fromId']
            .nunique()
        )
        .reset_index()
        .drop(columns='month')
        .groupby('slug')['fromId'].mean()
        .rename('Total Developers - Avg Last 6 Months')
    )
)

In [76]:
results = pd.concat([
    first_pr, 
    active_repos, 
    stars, forks, 
    repo_activity, 
    new_contribs, 
    lifetime_contribs, 
    dev_types
], axis=1).fillna(0).applymap(lambda x: round(x,1)).rename(index=slugs_to_names)
results.to_csv("octant-epoch-02-summary.csv")
results

,First PR Created - Years Ago,Active Repos - Last 6 Months,Max Stars - Any Repo,Max Forks - Any Repo,"Commit Code - All Repos, Last 6 Months","Issue Closed - All Repos, Last 6 Months","Issue Created - All Repos, Last 6 Months","Pull Request Created - All Repos, Last 6 Months","Pull Request Merged - All Repos, Last 6 Months",New Contributors - Last 6 Months,Total Contributors - All Time,Full-time Developers - Avg Last 6 Months,Part-time Developers - Avg Last 6 Months,Total Developers - Avg Last 6 Months
slug,,,,,,,,,,,,,,
BrightID,5.7,6.0,217.0,63.0,193.0,5.0,13.0,23.0,16.0,2.0,60,0.0,3.8,3.8
clr.fund,3.9,2.0,177.0,97.0,148.0,1.0,1.0,29.0,29.0,0.0,31,0.3,1.7,2.0
Token Engineering Commons,3.2,8.0,10.0,11.0,54.0,4.0,11.0,6.0,6.0,3.0,53,0.2,2.5,2.3
DAO Drops,2.1,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0
Drips,2.5,8.0,55.0,15.0,688.0,1.0,0.0,3.0,3.0,2.0,22,1.3,5.7,6.0
Ethereum Cat Herders,8.2,6.0,12264.0,5348.0,607.0,156.0,142.0,681.0,505.0,64.0,778,0.0,71.8,61.6
EthStaker,3.4,5.0,382.0,160.0,532.0,34.0,35.0,400.0,388.0,15.0,115,1.2,10.0,9.6
Gitcoin,6.3,43.0,1751.0,881.0,2699.0,1447.0,1514.0,1409.0,1222.0,27.0,555,10.2,26.7,31.6
Praise,2.1,6.0,31.0,16.0,345.0,60.0,68.0,40.0,37.0,2.0,29,0.3,2.5,2.8


In [71]:
results.to_csv("octant-epoch-02-summary.csv", sep="|")

In [72]:
(
    results[['First PR Created - Years Ago',
         'Total Contributors - All Time',
         'Max Forks - Any Repo',
         'Max Stars - Any Repo']]
    .sort_values(by='First PR Created - Years Ago', ascending=False)
)#.to_csv("octant-epoch-02-table1.csv", sep='|')

In [73]:
(
    results[[
        'New Contributors - Last 6 Months',
        'Full-time Developers - Avg Last 6 Months',
        'Part-time Developers - Avg Last 6 Months',
        'Total Developers - Avg Last 6 Months'
    ]]
    .sort_values(by='Total Developers - Avg Last 6 Months', ascending=False)
)#.to_csv("octant-epoch-02-table2.csv", sep='|')

In [74]:
(
    results[[
        'Issue Created - All Repos, Last 6 Months',
        'Issue Closed - All Repos, Last 6 Months',
        'Pull Request Created - All Repos, Last 6 Months',
        'Pull Request Merged - All Repos, Last 6 Months',
        'Commit Code - All Repos, Last 6 Months'        
    ]]
    .sort_values(by='Commit Code - All Repos, Last 6 Months', ascending=False)
    .applymap(int)
)#.to_csv("octant-epoch-02-table3.csv", sep='|')